In [ ]:
# Kich ban 3: StandardScaler with RandomizedSearchCV
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import to_categorical

# Load the train and test datasets
train_df = pd.read_csv('mitbih_train.csv')
test_df = pd.read_csv('mitbih_test.csv')

# Split the train and test datasets into X and y
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values
X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

# Preprocess the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for 1D-CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Convert labels to categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create the 1D-CNN model
def create_model():
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Conv1D(128, kernel_size=3, activation='relu'))
    model.add(Conv1D(128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Conv1D(256, kernel_size=3, activation='relu'))
    model.add(Conv1D(256, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create the KerasClassifier wrapper
model = KerasClassifier(build_fn=create_model)

param_grid = {
    'batch_size': [32, 64, 128],
    'epochs': [50, 100, 150]
}

# Perform random search to find the best hyperparameters
random_search = RandomizedSearchCV(model, param_grid, cv=3)
random_search.fit(X_train, y_train)

# Get the best model and its hyperparameters
best_model = random_search.best_estimator_
best_params = random_search.best_params_

# Print the best hyperparameters
print('Best Hyperparameters:', best_params)

# Train the best model
best_model.fit(X_train, y_train)

# Evaluate the best model on the test dataset
_, accuracy = best_model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

# Predict classes for the test dataset
y_test_pred = np.argmax(best_model.predict(X_test), axis=1)
y_test_true = np.argmax(y_test, axis=1)

# Generate classification report and Matthews correlation coefficient
report = classification_report(y_test_true, y_test_pred, target_names=['N', 'S', 'V', 'F', 'Q'])
mcc = matthews_corrcoef(y_test_true, y_test_pred)
print('Classification Report:\n', report)
print('Matthews Correlation Coefficient:', mcc)

# Calculate the confusion matrix
cm = confusion_matrix(y_test_true, y_test_pred)

# Map labels to their corresponding values
label_map = {0: 'N', 1: 'S', 2: 'V', 3: 'F', 4: 'Q'}
label_names = [label_map[i] for i in range(len(label_map))]

# Calculate the normalized confusion matrix
normalized_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Print the normalized confusion matrix
print('Normalized Confusion Matrix:')
print(normalized_cm)

# Plot the normalized confusion matrix
sns.heatmap(normalized_cm, annot=True, cmap='Blues', fmt='.4f')
plt.title('Normalized Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(np.arange(len(label_names)) + 0.5, label_names)
plt.yticks(np.arange(len(label_names)) + 0.5, label_names)
plt.show()